In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# EPFL Machine Learning Higgs

TODO description

## 1. Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
from implementations import *
from cross_validation import *

In [4]:
DATA_TRAIN_PATH = 'data/train.csv' 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## 2. Data pre-processing

Before any attempt to prediction, we perform some data cleaning (TODO) and preprocessing. We start by standardising our training dataset.

In [5]:
tx = standardise(tX)
# TODO proof of success ?

In [6]:
#### testing answer

kwargs = {}
kwargs['initial_w'] = np.random.rand(30)
kwargs['max_iters'] = 50
kwargs['gamma'] = 0.1
kwargs['lambda_'] = 0.1
kwargs['degree'] = 1

cross_validation(y, tx, compute_accuracy, 'least_squares', k_fold=6, **kwargs)

(0.8819890191348475,
 0.38894981267444556,
 0.7184474951599226,
 0.7184474951599226)

In [7]:
build_poly(tx[:,1], 3).shape

(250000, 4)

## 3. Classic ML methods

The following predictions are made using the functions implemented during the course labs.

### 3.1 Least Squares GD

In [8]:
initial_w = np.random.rand(30)
w, loss = least_squares_GD(y, tx, initial_w, 50, 0.1)

## Generate predictions and save ouput in csv format for submission:

In [9]:
DATA_TEST_PATH = 'data/test.csv'

# TODO standardise test data when required
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [10]:
OUTPUT_PATH = 'data/output.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(w, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [11]:
compute_accuracy(w, y, tx)

0.6973